In [1]:
import os
import json
import re

In [2]:
def merge_details(file_path):
    with open(file_path,'r',encoding='utf-8') as f:
        forward_details = f.readlines()


    merged_list = []
    current_item = ""

    for i, item in enumerate(forward_details):
        if item.strip() and item.strip()[0].isdigit():
            j = 1
            while j < len(item.strip()) and item.strip()[j].isdigit():
                j += 1
            if j < len(item.strip()) and item.strip()[j] == "：":
                merged_list.append(current_item)
                current_item = item.strip()
            else:
                current_item += item.strip()
        else:
            current_item += item.strip()

    merged_list.append(current_item)
    merged_list.pop(0)
    return merged_list


def get_extend_map(merged_list, mode):
    entity_extend_map = {}
    for i, item in enumerate(merged_list):
        if mode == "zh":
            item = item.replace(" ","")

        golds = eval(item.split("gold:")[1])
        for gold in golds:
            if gold not in entity_extend_map:
                entity_extend_map[gold] = []


        extend_ent_str = item.split("gold:")[0]
        matches = re.findall(r'\[(.*?)\]', extend_ent_str)

        for match in matches:
            if mode == "zh":
                match = match.replace("，",",").split(",")
            elif mode == "en":
                match = match.split(",")
            elif "zh" in mode:
                match = match.replace('"','').replace('（','').replace('）','').replace('(','').replace(')','')
                match = re.sub(r"[\[\]']", '', match.replace("，",",")).split(",")
            elif 'en' in mode:
                match = match.replace('"','').replace('（','').replace('）','').replace('(','').replace(')','')
                match = re.sub(r"[\[\]']", '', match.replace("，",",")).split(",")

            if len(match) != len(golds):
                continue
            for idx, extend_ent in enumerate(match):
                extend_ent = extend_ent.strip()
                if extend_ent not in entity_extend_map[golds[idx]]:
                    entity_extend_map[golds[idx]].append(extend_ent)

    lengths = [len(entity_extend_map[key]) for key in entity_extend_map]

    with open(f'./results/entity_extend_map_{mode}.json','w',encoding='utf-8') as f:
        json.dump(entity_extend_map, f, ensure_ascii=False, indent=4)
    
    return entity_extend_map, lengths


def get_filtered_extend_map(entity_extend_map, mode):
    if "_zh" in mode:
        gold_type = ['药物','身体','医疗程序','临床表现','医疗设备','医学检验项目','科室','微生物类','疾病']
    else:
        gold_type = ["organization", "person", "geographical social political", "vehicle", "location", "weapon", "facility"]

    
    filtered_entity_extend_map = {}
    for key in entity_extend_map:
        filtered_entity_extend_map[key] = [item for item in entity_extend_map[key] if item not in gold_type]

    filtered_lengths = [len(filtered_entity_extend_map[key]) for key in filtered_entity_extend_map]

    with open(f'./results/filtered_entity_extend_map_{mode}.json','w',encoding='utf-8') as f:
        json.dump(filtered_entity_extend_map, f, ensure_ascii=False, indent=4)

    return filtered_entity_extend_map, filtered_lengths



def human_process(filtered_entity_extend_map, mode):
    repeated_values = []
    value_to_keys = {}


    for key1, values1 in filtered_entity_extend_map.items():
        for key2, values2 in filtered_entity_extend_map.items():
            if key1 != key2 and key1 < key2:
                for value in values1:
                    if value in values2:
                        repeated_values.append(value)
                        if value in value_to_keys.keys():
                            if key1 not in value_to_keys[value]:
                                value_to_keys[value].append(key1)
                            if key2 not in value_to_keys[value]:
                                value_to_keys[value].append(key2)
                        else:
                            value_to_keys[value] = [key1, key2]

    with open(f'./results/交集_{mode}_raw.txt','w',encoding='utf-8') as f:
        for value, keys in value_to_keys.items():
            f.write('{} ：{}\n'.format(value, '、'.join(keys)))


In [3]:
def get_final_map(mode):
    with open(f'./results/filtered_entity_extend_map_{mode}.json','r',encoding='utf-8') as f:
        filtered_entity_extend_map_zh = json.load(f)

    with open(f'./results/交集_{mode}_raw.txt','r',encoding='utf-8') as f:
        raw_zh = f.readlines()

    tmp_dict1 = {item.split(' ：')[0].strip(): item.split(' ：')[1].strip().split('、') for item in raw_zh}
    jiaoji_raw_dict = {value: [key for key, val in tmp_dict1.items() if value in val] for key, values in tmp_dict1.items() for value in values}

    with open(f'./results/交集_{mode}.txt','r',encoding='utf-8') as f:
        new_zh = f.readlines()

    tmp_dict2 = {item.split(' ：')[0].strip(): item.split(' ：')[1].strip() for item in new_zh}
    jiaoji_new_dict = {}
    for key, value in tmp_dict2.items():
        if value in jiaoji_new_dict:
            jiaoji_new_dict[value].append(key)
        else:
            jiaoji_new_dict[value] = [key]

    for key in filtered_entity_extend_map_zh.keys():
        tmp_list = jiaoji_raw_dict[key]
        wuguan_list = [value for value in filtered_entity_extend_map_zh[key] if value not in tmp_list]
        ok_list = jiaoji_new_dict[key]
        filtered_entity_extend_map_zh[key] = wuguan_list + ok_list

    with open(f'./results/final_entity_extend_map_{mode}.json','w',encoding='utf-8') as f:
        json.dump(filtered_entity_extend_map_zh, f, ensure_ascii=False, indent=4)



In [4]:
class Process_details:
    def __init__(self, file_path, mode):
        self.file_path = file_path
        self.mode = mode


    def process(self):
        merged_list = merge_details(self.file_path)
        entity_extend_map, lengths = get_extend_map(merged_list, mode=self.mode)
        filtered_entity_extend_map, filtered_lengths = get_filtered_extend_map(entity_extend_map, mode=self.mode)
        human_process(filtered_entity_extend_map, mode=self.mode)

        return lengths, filtered_lengths
    
    def post_process(self):
        get_final_map(mode=self.mode)
        

CMeEE

In [ ]:
process_cmeee = Process_details('./results/extend_forward_zh.txt','zh')
lengths, filtered_lengths = process_cmeee.process()

ACE05

In [ ]:
process_ace05 = Process_details('./results/extend_forward_en.txt','en')
lengths, filtered_lengths = process_ace05.process()

Alpaca-CMeEE

In [ ]:
process_alpaca_cmeee = Process_details('./results/extend_forward_alpaca_33B_zh.txt','alpaca_33B_zh')
lengths, filtered_lengths = process_alpaca_cmeee.process()

Alpaca-ACE05

In [ ]:
process_alpaca_ace05 = Process_details('./results/extend_forward_alpaca_33B_en.txt','alpaca_33B_en')
lengths, filtered_lengths = process_alpaca_ace05.process()

Llama2-ACE05

In [17]:
process_llama2_ace05 = Process_details('./results/extend_forward_llama2_70B_en.txt','llama2_70B_en')
lengths, filtered_lengths = process_llama2_ace05.process()

ChatGLM-CMeEE

In [ ]:
process_chatglm_cmeee = Process_details('./results/extend_forward_chatglm_6B_zh.txt','chatglm_6B_zh')
lengths, filtered_lengths = process_chatglm_cmeee.process()

ChatGLM-ACE05

In [ ]:
process_chatglm_ace05 = Process_details('./results/extend_forward_chatglm_6B_en.txt','chatglm_6B_en')
lengths, filtered_lengths = process_chatglm_ace05.process()

GPT4-CMeEE

In [ ]:
process_gpt4_cmeee = Process_details('./results/extend_forward_gpt4_zh.txt','gpt4_zh')
lengths, filtered_lengths = process_gpt4_cmeee.process()

GPT4-ACE05

In [ ]:
process_gpt4_ace05 = Process_details('./results/extend_forward_gpt4_en.txt','gpt4_en')
lengths, filtered_lengths = process_gpt4_ace05.process()

Baichuan2-CMeEE

In [ ]:
process_baichuan2_cmeee = Process_details('./results/extend_forward_baichuan2_13B_zh.txt','baichuan2_13B_zh')
lengths, filtered_lengths = process_baichuan2_cmeee.process()

Baichuan2-ACE05

In [ ]:
process_baichuan2_ace05 = Process_details('./results/extend_forward_baichuan2_13B_en.txt','baichuan2_13B_en')
lengths, filtered_lengths = process_baichuan2_ace05.process()

In [9]:
# process_cmeee.post_process()
# process_ace05.post_process()
# process_alpaca_cmeee.post_process()
# process_alpaca_ace05.post_process()
# process_llama2_ace05.post_process()
# process_chatglm_cmeee.post_process()
# process_chatglm_ace05.post_process()
process_baichuan2_cmeee.post_process()
process_baichuan2_ace05.post_process()